In [1]:
import pandas as pd

In [2]:
# Open file    
fileHandler = open ("SMSSpamCollection.txt", "r", encoding="utf-8")
# Get list of all lines in file
listOfLines = fileHandler.readlines()
# Close file 
fileHandler.close()

In [3]:
df = pd.DataFrame(columns = ['text', 'label'])
df

,text,label


In [4]:
# Iterate over the lines
for line in listOfLines:
    spl = line.strip().split('\t')
    df = df.append({'label': spl[0], 'text': spl[1]}, ignore_index=True)
df

,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5569,This is the 2nd time we have tried 2 contact u...,spam
5570,Will ü b going to esplanade fr home?,ham
5571,"Pity, * was in mood for that. So...any other s...",ham
5572,The guy did some bitching but I acted like i'd...,ham


In [5]:
#Map applies a function to all the items in an input list or df column.
df['label'] = df.label.map({'ham':0, 'spam':1})
df.head()

,text,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


# Здесь текст чистим от мусора

In [19]:
import string #punctuation
import pprint
from collections import Counter #frequencies

def get_clean_text(text):
    #Bag of Words from scratch
    documents = text

    lower_case_documents = []

    for i in documents:
        lower_case_documents.append(i.lower())

    # Remove punctuation.
    sans_punctuation_documents = []

    for i in lower_case_documents:
        sans_punctuation_documents = ["".join( j for j in i if j not in string.punctuation) for i in  lower_case_documents]

    #Break each word
    preprocessed_documents = []

    for i in sans_punctuation_documents:
        preprocessed_documents.append(i.split(' ')) #split on space

    #Count frequency of words using counter
    frequency_list = []

    for i in preprocessed_documents:
        frequency_counts = Counter(i)
        frequency_list.append(frequency_counts)
    
    return preprocessed_documents, frequency_list, sans_punctuation_documents

In [7]:
df.to_excel('dataset.xlsx')

# Загружаем данные и чистим их

In [8]:
df = pd.read_excel('dataset.xlsx')

In [20]:
preprocessed_documents, frequency_list, sans_punctuation_documents = get_clean_text(df['text'])

# Разбиваем на трейн и тест

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sans_punctuation_documents,
                                                    df['label'],
                                                    random_state=1)

print("Our original set contains", df.shape[0], "observations")
print("Our training set contains", len(X_train), "observations")
print("Our testing set contains", len(X_test), "observations")

Our original set contains 5574 observations
Our training set contains 4180 observations
Our testing set contains 1394 observations


# Превращаем текст в вектора

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer() #set the variable

train = count_vector.fit_transform(X_train)

test = count_vector.transform(X_test)

# Обучаем модель, предсказываем тест, смотрим на результаты

In [39]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB() #call the method
naive_bayes.fit(train, y_train) #train the classifier on the training set

MultinomialNB()

In [40]:
predictions = naive_bayes.predict(test) #predic using the model on the testing set

In [42]:
from sklearn.metrics import accuracy_score, precision_score,f1_score

print('accuracy score: {}'.format(accuracy_score(y_test,predictions)))
print('precision score: {}'.format(precision_score(y_test,predictions)))

accuracy score: 0.9870875179340028
precision score: 0.9712643678160919


In [46]:
import pickle
# сохраняем модель в файл
filename = '11-708_Ivanaevskii_5.pkl'

with open(filename, 'wb') as fout:
    pickle.dump((count_vector, naive_bayes), fout)

# Вот отсюда надо вам тоже запускать блокнот для проверки. Тут загружаем модель и векторайзер для интерпретации. Методы чистки текста и превращения в вектор

In [3]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import string #punctuation
import pprint
from collections import Counter #frequencies

with open('11-708_Ivanaevskii_5.pkl', 'rb') as f:
    count_vector, loaded_model = pickle.load(f)
    
def get_clean_text(text):
    #Bag of Words from scratch
    documents = text

    lower_case_documents = []

    for i in documents:
        lower_case_documents.append(i.lower())

    # Remove punctuation.
    sans_punctuation_documents = []

    for i in lower_case_documents:
        sans_punctuation_documents = ["".join( j for j in i if j not in string.punctuation) for i in  lower_case_documents]

    #Break each word
    preprocessed_documents = []

    for i in sans_punctuation_documents:
        preprocessed_documents.append(i.split(' ')) #split on space

    #Count frequency of words using counter
    frequency_list = []

    for i in preprocessed_documents:
        frequency_counts = Counter(i)
        frequency_list.append(frequency_counts)
    
    return preprocessed_documents, frequency_list, sans_punctuation_documents

def get_predict(text):
    preprocessed_documents, frequency_list, sans_punctuation_documents = get_clean_text([text])
    
    test = count_vector.transform(sans_punctuation_documents)
    
    result = loaded_model.predict(test)
    return result

# Здесь вот проверяем свой текст в теле метода

In [4]:
r = get_predict('Free entry in 2 a wkly comp to win FA Cup')

# Тут ответ! 1 - спам, 0 - не спам!

In [5]:
r[0]

1